In [1]:
import pandas as pd
import os

# Ruta de la carpeta que contiene los archivos
carpeta = "C:/Users/Lenovo/Desktop/tesis/repositorio/hydro_generation/data"

# Lista para almacenar los DataFrames
dfs = []

# Iterar sobre cada archivo en la carpeta
for archivo in os.listdir(carpeta):
    # Construir la ruta completa del archivo
    ruta_archivo = os.path.join(carpeta, archivo)
    
    # Leer el archivo y agregarlo a la lista de DataFrames
    try:
        df = pd.read_csv(ruta_archivo, sep='\t')  # Asumiendo que el separador es una tabulación
        dfs.append(df)
    except Exception as e:
        print(f"Error al leer el archivo {archivo}: {e}")

# Concatenar todos los DataFrames en uno solo
df_final = pd.concat(dfs, ignore_index=True)

# Mostrar las primeras filas del DataFrame final
print(df_final.head())


  fecha_opreal  hora_opreal  central_infotecnica_id  \
0   2016-01-01            1                      47   
1   2016-01-01            1                     241   
2   2016-01-01            1                     218   
3   2016-01-01            1                     278   
4   2016-01-01            1                      53   

              central_nombre central_tipo central_tipo_nemotecnico  \
0             PMGD TER TIRÚA      Térmica                  termica   
1         HP HIDROBONITO MC1   Hidráulica               hidraulica   
2       TER ENERGÍA PACÍFICO      Térmica                  termica   
3     PMGD PFV SANTA CECILIA        Solar                    solar   
4  PMGD TER LOUSIANA PACIFIC      Térmica                  termica   

   generacion_real_mwh  generacion_real_ernc_mwh  
0                 0.00                      0.00  
1                 2.79                      2.79  
2                 0.00                      0.00  
3                 0.00                      

In [2]:
# Filtrando el DataFrame por las condiciones especificadas
df_filtrado = df_final[(df_final['central_nombre'].str.startswith('HE')) & 
                       (df_final['central_tipo'] == 'Hidráulica')]

In [3]:
# Filtrando nuevamente el DataFrame para excluir específicamente las filas con los nombres 'HE SAUZAL' y 'HE PILMAIQUÉN'
df_filtrado_correcto = df_filtrado[~df_filtrado['central_nombre'].isin(['HE SAUZAL', 'HE PILMAIQUÉN'])]

df_filtrado_correcto

,fecha_opreal,hora_opreal,central_infotecnica_id,central_nombre,central_tipo,central_tipo_nemotecnico,generacion_real_mwh,generacion_real_ernc_mwh
18,2016-01-01,1,178,HE PANGUE,Hidráulica,hidraulica,53.0,0.0
28,2016-01-01,1,121,HE EL TORO,Hidráulica,hidraulica,218.0,0.0
56,2016-01-01,1,128,HE RAPEL,Hidráulica,hidraulica,0.0,0.0
61,2016-01-01,1,181,HE PEHUENCHE,Hidráulica,hidraulica,221.0,0.0
82,2016-01-01,1,90,HE CANUTILLAR,Hidráulica,hidraulica,56.0,0.0
...,...,...,...,...,...,...,...,...
40480791,2023-09-30,20,92,HE COLBUN,Hidráulica,hidraulica,458.0,0.0
40480792,2023-09-30,21,92,HE COLBUN,Hidráulica,hidraulica,474.0,0.0
40480793,2023-09-30,23,92,HE COLBUN,Hidráulica,hidraulica,474.0,0.0
40480794,2023-09-30,24,92,HE COLBUN,Hidráulica,hidraulica,474.0,0.0


In [4]:
import pandas as pd

# Convirtiendo la columna 'fecha_opreal' a datetime y ordenando el DataFrame
df_filtrado_correcto['fecha_opreal'] = pd.to_datetime(df_filtrado_correcto['fecha_opreal'])
df_filtrado_ordenado = df_filtrado_correcto.sort_values(by='fecha_opreal')

# Creando un nuevo DataFrame con la suma diaria de la generación hidráulica
df_suma_diaria = df_filtrado_ordenado.groupby(df_filtrado_ordenado['fecha_opreal'].dt.date)['generacion_real_mwh'].sum().reset_index()
df_suma_diaria.rename(columns={'fecha_opreal': 'fecha', 'generacion_real_mwh': 'generacion_diaria_hidraulica'}, inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17728\139130057.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_correcto['fecha_opreal'] = pd.to_datetime(df_filtrado_correcto['fecha_opreal'])


In [5]:
df_suma_diaria

,fecha,generacion_diaria_hidraulica
0,2016-01-01,22310.00000
1,2016-01-02,36071.00000
2,2016-01-03,35736.00000
3,2016-01-04,50188.00000
4,2016-01-05,52281.00000
...,...,...
2917,2023-12-27,60854.40569
2918,2023-12-28,60033.38493
2919,2023-12-29,55326.49000
2920,2023-12-30,55031.43000


In [6]:
import plotly.express as px

# Asegúrate de que df_suma_diaria es tu DataFrame
# Creando un line plot con Plotly para visualizar la serie de tiempo de generación diaria hidráulica
fig = px.line(df_suma_diaria, x='fecha', y='generacion_diaria_hidraulica', title='Serie de Tiempo de Generación Diaria Hidráulica')

# Mostrando el gráfico
fig.show()





# Análisis Descriptivo de la Generación Hidroeléctrica Diaria

La serie de tiempo que representa la generación hidroeléctrica diaria muestra patrones interesantes que varían antes y después del año 2017. Al observar los datos, se notan algunas tendencias y características clave:

### Antes de 2017

Alta Dispersión: Los datos de generación diaria antes del 2017 muestran una dispersión significativa. Esto indica una variabilidad considerable en la generación hidroeléctrica de un día a otro.

Dificultad para Discernir Tendencias: Debido a esta alta variabilidad, es difícil identificar tendencias claras y consistentes en este período. Las fluctuaciones diarias pueden deberse a una variedad de factores, como condiciones climáticas variables, mantenimientos de las instalaciones, o cambios en la demanda de energía.

### Después de 2017

Tendencias Más Claras: A partir del 2017, la serie de tiempo muestra patrones más coherentes y menos dispersión. Esto sugiere una mayor estabilidad en la generación hidroeléctrica diaria.
Visualización de Tendencias: Con esta menor variabilidad, se hace más fácil visualizar y seguir las tendencias a lo largo del tiempo. Esto puede reflejar una mejor gestión de los recursos hídricos, una mayor eficiencia en la generación de energía, o posiblemente un clima más estable.

### Implicaciones y Reflexiones
Gestión de Recursos Hídricos: La estabilización observada en la generación hidroeléctrica después de 2017 podría indicar mejoras en la gestión de los recursos hídricos y en la operación de las centrales hid
roeléctricas.

### Impacto del Cambio Climático:
 Los cambios en el patrón de generación también pueden estar influenciados por el cambio climático, que puede afectar la disponibilidad y regularidad de los recursos hídricos.
Planificación y Estrategias Futuras: Comprender estas tendencias es crucial para la planificación futura en términos de capacidad de generación hidroeléctrica, especialmente en el contexto del cambio climático y de la creciente demanda de energía.

### Conclusión

El análisis de la generación hidroeléctrica diaria revela información valiosa sobre la estabilidad y eficiencia del sector hidroeléctrico. La comparación entre los períodos antes y después de 2017 destaca la importancia de la gestión eficiente de los recursos y la adaptación a los desafíos ambientales y operativos. Continuar monitorizando estas tendencias ayudará a tomar decisiones informadas para el futuro del sector energético.